# Energy Consumption and CO2 Emissions Analysis
## Statistical Analysis

**Prerequisites:**
- Completed notebook 01_data_exploration_and_cleaning.ipynb
- Cleaned dataset: cleaned_energy_data.csv

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

## Load Cleaned Dataset

In [2]:
from pathlib import Path
import os

current_dir = Path.cwd()
parent = current_dir.parent

os.chdir(parent)
current_dir = str(Path.cwd())   # update the variable so future code is consistent
print("New current directory:", current_dir)
processed_file_path = current_dir+'\\dataset\\processed\\cleaned_energy_data.csv'
df = pd.read_csv(processed_file_path)

New current directory: d:\Code Institute\Energy-Consumption-CO2-Emissions-Analysis


In [3]:
print(f"Dataset loaded: {len(df):,} records")
print(f"Columns: {len(df.columns)}")
print(f"\nDate range: {df['Year'].min()} - {df['Year'].max()}")
print(f"Countries: {df['Country'].nunique()}")
print(f"Energy categories: {df['Energy_category'].nunique()}")

Dataset loaded: 46,200 records
Columns: 16

Date range: 1980 - 2019
Countries: 231
Energy categories: 5
